# Optimization and Gradient Descent

Optimization is a broad topic in machine learning; we will focus on a specific (but large and powerful) subset of optimization in which an algorithm attempts to learn the parameters of a function that minimize its (continuous real-valued) output. If one can perform this kind of optimization, one can also use it to learn the parameters of a model that best explains a set of data. PyTorch excels at both of these methods, which are the foundation of the next lesson on neural networks. We'll start by examining the former then use it to construct the latter method in the next section.

## How does gradient descent optimization work?

The basic idea of gradient descent is that the gradient of a function points in the direction it is increasing the fastest and its opposite is the direction in which it decreases the fastest, so if you make a guess at the parameters that minimize the function then calculate the gradient at that point, it tells you the direction of a point that's closer to the minimum. You can then take a small step in that direction and repeat the process until you're as close as you need to be.

We'll unpack the description of gradient descent immediately above using a simple example. Suppose we are looking for the minimum of the following function:

$$ f(x, y) = (x - 1)^2 + 4(y + 1)^2 + \frac{x y}{2} $$

This function doesn't mean anything to us in particular, but we'll use it as an example. In this example, $f$ is our *loss* function&mdash;i.e., the real-valued continuous function that we are trying to minimize. Here, $x$ and $y$ are the parameters that we are learning.

## Example: finding the minimum of a simple function.

We'll start by implementing the function $f$ (from above) in Python code.

In [ ]:
import torch

# This function will work fine with PyTorch tensors or with NumPy arrays
# as long as both x and y are the same type.
def f(x, y):
    return (x - 1)**2 + 4*(y + 1)**2 + x*y/2

# Test that it works:
f(3, 3)

Notice that we can make tensors out of individual numbers (individual numbers are just rank-0 tensors), and when we perform calculations using these tensor numbers, they yield tensor numbers as well.

In [ ]:
x = torch.tensor(3.0)
y = torch.tensor(3.0)

f(x, y)

### The `requires_grad` option enables gradient calculation.

Recall that the `torch.tensor` function and similar tensor-creation functions like `torch.ones` and `torch.zeros` accept an optional argument `requires_grad`. This option tells a tensor that it's a parameter to a function involved in some kind of optimization, and so the gradient of this value with respect to some computed value may be required. Let's look at an example of this.

In [ ]:
x = torch.tensor(3.0, requires_grad=True)
y = torch.tensor(3.0, requires_grad=True)

# Calculate the value of the function:
z = f(x, y)

# Ask PyTorch to backward-propagate the gradient of z to its parameters
# (this calculates the gradient of z with respect to its parameters x and y
# then tells those parameters what their components of the gradients are).
z.backward()

# Now we can examine the gradient of f(x,y) with respect to x and y:
print('∂f/∂x =', x.grad)
print('∂f/∂y =', y.grad)

The `requires_grad` option can be very powerful and is required for all parameters in PyTorch optimizations. It does induce a few changes to the way that that one interacts with the tensor, however. These changes are required because PyTorch needs to carefully keep track of all the calculations that result from any tensors that require gradients&mdash;if a tensor involved in these calculations gets updated outside of PyTorch's ecosystem, it can lose track of critical parts of the calculation and create incorrect gradients.

Primarily, when `requires_grad` is enabled, accessing the NumPy array of a PyTorch tensor requires an extra step, and in-place operations become illegal.

In [ ]:
# For normal tensors of single numbers, we can update the tensor by using an
# empty tuple in the setitem paradigm (tensor[()] = new_value):
tens = torch.tensor(0.0)
tens[()] = 1.0
print('tens:', tens)

# But for a tensor that requires gradient, one cannot update the tensor
# directly because this would ruin the gradient tracking.
x[()] = 10  # This will raise an error.

In [ ]:
# For a normal tensor, we can access it's NumPy array using the x.numpy()
# method.
tens_np = tens.numpy()
print('tens_np:', tens_np)

# For a tensor that requires gradient, this will raise an error, because
# editing the NumPy tensor would ruin the gradient tracking (and once PyTorch
# returns the array, it can't stop you from editing it).
print(x.numpy())  # This will raise an error.

In [ ]:
# The same is true of any tensor involved in gradient tracking; because z is
# the result of a computation that included a tensor with requires_grad=True,
# z cannot be directly accessed or edited either.
print(z.numpy())  # This will raise an error.

In [ ]:
# As the error message suggests, we have to detach a tensor that is part of
# any gradient tracking before we extract a numpy array.
print(x.detach().numpy())

The `detach()` method can be used to make a duplicate PyTorch tensor that no longer has `requires_grad` set to `True`.

```{warning}
Although `x.detach()` returns a new PyTorch tensor that has been detached from the gradient tracking system, it still uses the same memory under the hood to store the tensor elements as the original PyTorch tensor. This means that if you edit the detached tensor or its associated NumPy array, it can produce errors when gradients are computed. If you plan to edit these arrays you should copy them first!
```

### The Optimization Loop

To perform the minimization, we'll start by making a guess as to $x$ and $y$ values that yield the minimum. (It may not be a very good guess.) We'll create an optimizer (a PyTorch class) that manages the state of the optimization. We'll then take repeated steps toward the minimum using the gradient. (PyTorch mostly does this automatically for us.) In each step, we'll perform a few sub-steps:
1. **Calculate the value of the function at $x$ and $y$ (`z = f(x, y)`).** The value `z` that is returned is a PyTorch tensor.
2. **Calculate and propogate the gradient backward from `z` to its parameters `x` and `y`.** PyTorch does this step for us via the method `z.backward()`.
3. **Tell the optimizer to take a step.** PyTorch does the work in this step, which involves updating all of the parameters (`x` and `y` in this case), to be a little closer to the minimum.

```{note}
The number of steps to take is a hyperparemter of the optimization. A larger number of steps will usually result in a better optimization, but the improvement in the optimization usually diminishes with each step.

Alternatively, one can use a heuristic to decide when to stop, such as choosing to take steps until the change in the function value is smaller than some fixed value.
```

In [ ]:
# There are a few hyperparameters we can declare ahead of time.

# The number of steps determins how many minimization steps we take.
n_steps = 50

# The learning rate is essentially a knob we can turn to try to speed up or
# slow down the optimization. A higher learning rate means that the optimizer
# takes larger steps along the gradient; a lower learning rate means that it
# takes smaller steps. Small steps converge more slowly, but if you take too
# large of a step you can pass the minimum and potentially get farther away.
# The best learning rate will depend on the optimizer and the function being
# optimized, so it often has to be found experimentally.
lr = 0.1

# Now that we've declared our hyperparameters, let's declare our parameters.
x = torch.tensor(3.0, requires_grad=True)
y = torch.tensor(3.0, requires_grad=True)

# Next, we can declare an optimizer.
# We'll use the optimizer SGD: stochastic gradient descent.
# The "stochastic" refers to the way it handles training when a dataset is
# involved and does not indicate that this method is stochastic in this case.
# The optimizer will manage the steps and updating the parameters during them.
# Because of this we have to tell it the parameters and the learning rate.
optimizer = torch.optim.SGD([x, y], lr=lr)

# Now we can take several optimization steps:
for step_number in range(n_steps):
    # We're starting a new step, so we reset the gradients.
    optimizer.zero_grad()
    # Calculate the function value at these parameters.
    z = f(x, y)
    # Have PyTorch backward-propagate the gradients.
    z.backward()
    # If the norm of the gradient is less than 1e-5, we finish early.
    if torch.hypot(x.grad, y.grad) < 1e-4:
        break
    # Print a message about this step:
    print("Step number", step_number)
    print("  x = ", float(x), ";  ∂f/∂x = ", float(x.grad))
    print("  y = ", float(y), ";  ∂f/∂y = ", float(y.grad))
    print("  z = ", float(z))
    # Have the optimizer take a step:
    optimizer.step()

# After the optimizer has run, print out what it's found:
print("Final result:")
print(f"  f({float(x)}, {float(y)}) = {float(z)}")

### Effects of the learning rate parameter.

We can try running the above code-block with different hyperparameters to get a sense of their impact. Try running the above block using a high learning rate (`lr=0.5`) and a low learning rate (`lr=0.01`).

With `lr=0.01`, the optimization works fine, but it doesn't finish. It gets closer to the minimum, but it doesn't reach the minimum value found with `lr=0.1`.

With `lr=0.5` the optimization actually diverges! This means that it takes steps so large that they go so past the minimum to a point that's higher than the start point.

To get a better sense for what these mean, let's make some plots of the steps that the optimizer takes. Try running this with different `lr` parameters.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Declare our hyperparameters:
n_steps = 50
lr = 0.1

# Now the parameters:
x = torch.tensor(3.0, requires_grad=True)
y = torch.tensor(3.0, requires_grad=True)

# Now the optimizer:
optimizer = torch.optim.SGD([x, y], lr=lr)

# Now we set up a pyplot figure.
(fig,ax) = plt.subplots(1, 1, figsize=(3,3), dpi=288)
fig.subplots_adjust(0, 0, 1, 1, 0, 0)
# Make an image of the function itself and plot it as the background.
(x_im, y_im) = np.meshgrid(
    np.linspace(-5,5,512),
    np.linspace(-5,5,512),
    indexing='xy')
z_im = f(x_im, y_im)
ax.imshow(z_im, cmap='jet', zorder=-1, extent=(-5,5,5,-5))
ax.invert_yaxis()

# Now we can take several optimization steps:
for step_number in range(n_steps):
    # We're starting a new step, so we reset the gradients.
    optimizer.zero_grad()
    # Calculate the function value at these parameters.
    z = f(x, y)
    # Have PyTorch backward-propagate the gradients.
    z.backward()
    # Plot the points and the gradients:
    (x_np, y_np) = (x.detach().numpy(), y.detach().numpy())
    (x_np, y_np) = (np.array(x_np), np.array(y_np))
    ax.plot(x_np, y_np, 'w.', ms=0.5)
    # If the norm of the gradient is less than 1e-5, we finish early.
    if torch.hypot(x.grad, y.grad) < 1e-4:
        break
    # Have the optimizer take a step:
    optimizer.step()
    # Plot the arrow from the previous to this point.
    dx_np = x.detach().numpy() - x_np
    dy_np = y.detach().numpy() - y_np
    ax.arrow(x_np, y_np, dx_np, dy_np, color='w', lw=0.25, head_width=0.06)

# After the optimizer has run, show the steps:
ax.set_xlim([-5,5])
ax.set_ylim([-5,5])
plt.show()

## Another Example: The problem of local minima.

Let's examine a different function. We'll use essentially the same optimization loop but will define a slightly different function:

$$ g(x, y) = \left((x + 1)^2 + (y + 1)^2\right) \left((x - 3)^2 + (y - 3)^2 + 1\right) $$

Let's defint this function then run our optimization again.

In [ ]:
def g(x, y):
    return ((x + 1)**2 + (y + 1)**2) * ((x - 3)**2 + (y - 3)**2 + 1)

In [ ]:
# Declare our hyperparameters:
n_steps = 50
lr = 0.005  # This function is different and needs a lower learning rate.

# Now the parameters; we'll start at slightly different position this time.
x = torch.tensor(3.5, requires_grad=True)
y = torch.tensor(4.0, requires_grad=True)

# Now the optimizer:
optimizer = torch.optim.SGD([x, y], lr=lr)

# Now we set up a pyplot figure.
(fig,ax) = plt.subplots(1, 1, figsize=(3,3), dpi=288)
fig.subplots_adjust(0, 0, 1, 1, 0, 0)
# Make an image of the function itself and plot it as the background.
(x_im, y_im) = np.meshgrid(
    np.linspace(-5,5,512),
    np.linspace(-5,5,512),
    indexing='xy')
z_im = g(x_im, y_im)
# We can add a vmax to make sure our visualization captures the part of the
# image that is of interest to us.
ax.imshow(z_im, cmap='jet', zorder=-1, extent=(-5,5,5,-5), vmax=500)
ax.invert_yaxis()

# Now we can take several optimization steps:
for step_number in range(n_steps):
    # We're starting a new step, so we reset the gradients.
    optimizer.zero_grad()
    # Calculate the function value at these parameters.
    z = g(x, y)
    # Have PyTorch backward-propagate the gradients.
    z.backward()
    # Plot the points and the gradients:
    (x_np, y_np) = (x.detach().numpy(), y.detach().numpy())
    (x_np, y_np) = (np.array(x_np), np.array(y_np))
    ax.plot(x_np, y_np, 'w.', ms=0.5)
    # If the norm of the gradient is less than 1e-5, we finish early.
    if torch.hypot(x.grad, y.grad) < 1e-4:
        break
    # Have the optimizer take a step:
    optimizer.step()
    # Plot the arrow from the previous to this point.
    dx_np = x.detach().numpy() - x_np
    dy_np = y.detach().numpy() - y_np
    ax.arrow(x_np, y_np, dx_np, dy_np, color='w', lw=0.25, head_width=0.06)

# After the optimizer has run, show the steps:
ax.set_xlim([-5,5])
ax.set_ylim([-5,5])
plt.show()

Clearly in the above example, the method found a minimum value, but there's another minimum value in this function, and it's lower than the minimum that the optimization method found. What if we change the initial parameters `x` and `y` to have a start value closer to the other minimum?

Local minima are an issue in many optimization problems. In some cases, it can be proven that a global minimum has been found, but in many cases it cannot be. One strategy for avoiding local minima is to fit a function many times and to use, as the final result, whatever lowest value was achieved across all runs. Of course, this is quite time consuming, and even with many random starts, there's no guarantee that the global minimum will be found. While a broader discussion of local minima is beyond the scope of this course, it is always important to think about the possibility of local minima in any optimization one is performing.